In [8]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
from transformers import AutoTokenizer, pipeline, logging

In [10]:
import random
import json
import torch
import gc

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("lmsys/vicuna-7b-v1.5", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.5")


model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/vvjain3/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/vvjain3/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
 

In [12]:
input_ids = tokenizer("""### Human: Given a question: Which of the following best describes the structure that collects urine in the body? and a correct answer: Bladder, is there a way to generate a word same as correct answer but not verbatim?
### Assistant:""", return_tensors="pt").input_ids
input_ids.shape, input_ids

(torch.Size([1, 58]),
 tensor([[    1,   835, 12968, 29901, 11221,   263,  1139, 29901,  8449,   310,
            278,  1494,  1900, 16612,   278,  3829,   393,  6314, 29879,  5065,
            457,   297,   278,  3573, 29973,   322,   263,  1959,  1234, 29901,
           3164,   328,   672, 29892,   338,   727,   263,   982,   304,  5706,
            263,  1734,  1021,   408,  1959,  1234,   541,   451,  9750,   271,
            326, 29973,    13,  2277, 29937,  4007, 22137, 29901]]))

In [13]:
dict1 = {'result': (1, None),
 'question': 'What is the location of death of joseph paterno?',
 'correct_answer': 'pennsylvania',
 'model_answer': ' [/INST] Joseph Paterno died in Pennsylvania, specifically of complications from lung cancer 74 days after',
 'context': 'Joseph Vincent Paterno (; December 21, 1926 - January 22, 2012), sometimes referred to as JoePa, was an American college football player, athletic director, and coach. He was the head coach of the Penn State Nittany Lions from 1966 to 2011. With 409 victories, Paterno is the most victorious coach in NCAA FBS history.  He recorded his 409th victory on October 29, 2011, then his career ended 11 days later with his dismissal from the team on November 9, 2011 as a result of  the Penn State child sex abuse scandal. He died 74 days later, of complications from lung cancer.Paterno was born in Brooklyn, New York. He attended Brown University, where he played football both ways as the quarterback and a cornerback. He had originally planned on going to law school, but he was instead hired in 1950 as an assistant football coach at Penn State. He was persuaded to do this by his college coach Rip Engle, who had taken over as Penn State\'s head coach. In 1966, Paterno was named as Engle\'s successor. He soon coached the team to two undefeated regular seasons in 1968 and 1969. The team won two national championships—in 1982 and 1986. Paterno coached five undefeated teams that won major bowl games, and in 2007 he was inducted into the College Football Hall of Fame as a coach. During his career, he led the Nittany Lions to 37 bowl appearances with 24 wins while turning down offers to coach National Football League (NFL) teams that included the Pittsburgh Steelers and the New England Patriots.After the child sex abuse scandal involving his former defensive coordinator Jerry Sandusky broke in full in November 2011, Paterno announced that he would retire at the end of the season. However, on November 9, the Penn State Board of Trustees rejected this disclosure and immediately terminated his contract. An investigation conducted by former FBI director Louis Freeh concluded in July 2012 that Paterno concealed information relating to Sandusky\'s sexual abuse of young boys. The investigation also uncovered information that Paterno may have persuaded university officials not to report Sandusky to authorities in 2001. A critique of the Freeh report composed by the law firm King & Spalding, which was commissioned by the Paterno family and included expert opinion from former U.S. attorney general and Pennsylvania governor Dick Thornburgh, among others, disputed Paterno\'s involvement in the alleged cover-up and accused Freeh of making unsupported conclusions. Freeh called the critique a "self-serving report" that "does not change the facts."On July 23, 2012, the NCAA vacated all of Penn State\'s wins from 1998 through 2011 as part of its punishment for the child sex abuse scandal. The association eliminated 111 of the games Paterno had coached and won, which dropped him from first to 12th on the list of winningest NCAA football coaches. In January 2013, State senator Jake Corman and state treasurer Rob McCord launched a lawsuit against the NCAA to overturn the sanctions on Penn State on the basis that Freeh had been actively collaborating with the NCAA and that due process had not been followed. As part of the settlement, the NCAA reversed its decision on January 16, 2015, and restored the 111 wins to Paterno\'s record.joseph vincent paterno is also known as joseph paterno.\nPennsylvania ( (listen) PEN-səl-VAY-nee-ə), officially the Commonwealth of Pennsylvania, is a state located in the northeastern, Great Lakes and Mid-Atlantic regions of the United States. The Appalachian Mountains run through its middle. The Commonwealth is bordered by Delaware to the southeast, Maryland to the south, West Virginia to the southwest, Ohio to the west, Lake Erie and the Canadian province of Ontario to the northwest, New York to the north, and New Jersey to the east.Pennsylvania is the 33rd-largest state by area, and the 5th-most populous state according to the most recent official U.S. Census count in 2010. It is the 9th-most densely populated of the 50 states. Pennsylvania\'s two most populous cities are Philadelphia (1,580,863), and Pittsburgh (302,407). The state capital and its 10th largest city is Harrisburg. Pennsylvania has 140 miles (225 km) of waterfront along Lake Erie and the Delaware Estuary.The state is one of the 13 original founding states of the United States; it came into being in 1681 as a result of a royal land grant to William Penn, the son of the state\'s namesake. Part of Pennsylvania (along the Delaware River), together with the present State of Delaware, had earlier been organized as the Colony of New Sweden. It was the second state to ratify the United States Constitution, on December 12, 1787. Independence Hall, where the United States Declaration of Independence and United States Constitution were drafted, is located in the state\'s largest city of Philadelphia. During the American Civil War, the Battle of Gettysburg was fought in the south central region of the state. Valley Forge near Philadelphia was General Washington\'s headquarters during the bitter winter of 1777–78.\nGalluccio is a comune (municipality) in the Province of Caserta in the Italian region Campania, located about 60 kilometres (37 mi) northwest of Naples and about 45 kilometres (28 mi) northwest of Caserta. It is at the feet of the southern slopes of Monte Camino.\nSaint-Clément-des-Levées is a commune in the Maine-et-Loire department in western France.\n',
 'path': ['joseph vincent paterno', 'pennsylvania']}

In [14]:
prompt = f"""### Human: context: {dict1['context']} \n ### Assistant:I understand the context. What can I help you with? \n\n Human:Using the above context, answer the query:{dict1['question']}. Start with the answer in one word or phrase. \n Assistant: """

In [20]:
decoded[0][decoded[0].index('Start with the answer in one word or phrase.')+len('Start with the answer in one word or phrase.') - 20:]

' one word or phrase. \n Assistant: \n\nHuman:What is the location of death of joseph paterno? \n'

In [122]:
def chat_apply(messages):
    system_prompt = """A chat between a curious user and an artificial intelligence assistant.
        The assistant gives helpful, detailed, and polite answers to the user's questions."""
    
    seps = [' ', '</s>']
    ret = system_prompt + seps[0]
    for i, (role, message) in enumerate(messages):
        role = messages[i]['role']
        message = messages[i]['content']
        if message:
            ret += role + ": " + message + seps[i % 2]
        else:
            ret += role + ":"
    return ret

In [154]:
messages = [
                {"role": "USER", "content": f"context: {dict1['context']}"},
                {"role": "ASSISTANT", "content": "I understand the context. What can I help you with?"},
                {"role": "USER", "content": f"Using the above context, answer the query: {dict1['question']}. Start with answer"},
            ]
prompt = chat_apply(messages)

In [103]:
messages1 = [
                {"role": "user", "content": f"context: {dict1['context']}"},
                {"role": "assistant", "content": "I understand the context. What can I help you with?"},
                {"role": "user", "content": f"Using the above context, answer the query: {dict1['question']}. Start with the answer in one word or phrase."},
            ]

In [149]:
prompt = """Given context: ICDC College was a private for-profit, post-secondary education career college. ICDC College had Four Southern California campuses (Van Nuys, South Bay, Hollywood, and Huntington Park) and an online campus.  The college closed in 2016 and Trident University International has agreed to continue instruction of ICDC's current academic programs.icdc college is also known as the day the earth stood still.
Twenty-First Century Fox, Inc., doing business as 21st Century Fox (21CF), was an American multinational mass media corporation that was based in Midtown Manhattan, New York City. It was one of the two companies formed from the 2013 spin-off of the publishing assets of News Corporation, as founded by Rupert Murdoch in 1980.21st Century Fox was the legal successor to News Corporation dealing primarily in the film and television industries. It was the United States' fourth-largest media conglomerate until its acquisition by The Walt Disney Company in 2019. The other company, the present-day News Corporation, holds Murdoch's print interests and other media assets in Australia (both owned by him and his family via a family trust with 39% interest in each). Murdoch was co-executive chairman, while his sons Lachlan Murdoch and James Murdoch were co-executive chairman and CEO, respectively.21st Century Fox's assets included the Fox Entertainment Group—owners of the 20th Century Fox film studio (the company's partial namesake), the Fox television network,  and a majority stake in National Geographic Partners—the commercial media arm of the National Geographic Society, among other assets. It also had significant foreign operations, including the prominent Indian television channel operator Star India. The company ranked No. 109 in the 2018 Fortune 500 list of the largest United States corporations by total revenue.On July 27, 2018, 21st Century Fox shareholders agreed to sell the majority of its assets to Disney for $71.3 billion. The sale covered the majority of 21CF's entertainment assets, including 20th Century Fox, FX Networks, and National Geographic Partners among others. Following a bidding war with Fox, Sky plc (a British media group which Fox held a stake in) was acquired separately by Comcast, while Fox's FSN regional sports networks was sold to Sinclair Broadcast Group to comply with antitrust rulings. The remainder, consisting primarily of the Fox and MyNetworkTV networks, and Fox's national Broadcasting, Television Stations, news and sports operations, were spun out into a new company, Fox Corporation, which began trading on March 19, 2019. Disney's acquisition of 21CF was closed on March 20 after which the remaining 21CF's assets were scattered across the divisions of Disney.
The Day the Earth Stood Still (a.k.a. Farewell to the Master and Journey to the World) is a 1951 American black-and-white science fiction film from 20th Century Fox, produced by Julian Blaustein and directed by Robert Wise. The film stars Michael Rennie, Patricia Neal, Billy Gray, Hugh Marlowe, and Sam Jaffe. The screenplay was written by Edmund H. North, based on the 1940 science fiction short story "Farewell to the Master" by Harry Bates, and the film score was composed by Bernard Herrmann.The storyline for The Day the Earth Stood Still involves a humanoid alien visitor named Klaatu that comes to Earth, accompanied by a powerful eight-foot tall robot, Gort, to deliver an important message that will affect the entire human race.In 1995, the film was selected for preservation in the United States National Film Registry as "culturally, historically, or aesthetically significant".
The original incarnation of  News Corporation (abbreviated News Corp.) was an American multinational mass media corporation operated and owned by media mogul, Rupert Murdoch, headquartered in New York City. Prior to its split in 2013, it was the world's fourth-largest media group in terms of revenue, and News Corporation had become a media powerhouse since its inception, almost dominating the news, television, film and print industries.News Corporation was a publicly traded company listed on NASDAQ. Formerly incorporated in Adelaide, South Australia, the company was re-incorporated under Delaware General Corporation Law after a majority of shareholders approved the move on November 12, 2004. News Corporation was headquartered at 1211 Avenue of the Americas, New York, in the newer 1960s–1970s corridor of the Rockefeller Center complex.On June 28, 2012, after concerns from shareholders in response to its recent scandals and to "unlock even greater long-term shareholder value", founder Rupert Murdoch announced that News Corporation's assets would be split into two publicly traded companies, one oriented towards media, and the other towards publishing. The corporate spin-off formally took place on June 28, 2013; where the present News Corp. was renamed 21st Century Fox and consists primarily of media outlets, while a new News Corp was formed to take on the publishing and Australian broadcasting assets.Its major holdings at the time of the split were News Limited (a group of newspaper publishers in Murdoch's native Australia), News International (a newspaper publisher in the United Kingdom, whose properties include The Times, The Sun, and the now-defunct News of the World—which was the subject of a phone hacking scandal that led to its closure in July 2011), Dow Jones & Company (an American publisher of financial news outlets, including The Wall Street Journal), the book publisher HarperCollins, and the Fox Entertainment Group (owners of the 20th Century Fox film studio and the Fox Broadcasting Company—one of the United States' major television networks).
Twentieth Century Fox Film Corporation (colloquial: Twentieth Century Fox; 20th Century Fox; 20th; Fox) is an American film studio that is a subsidiary of Walt Disney Studios, a division of The Walt Disney Company. The studio is located on the Fox lot in the Century City area of Los Angeles.For over 83 years, it was one of the "Big Six" major American film studios; formed from the merger of the Fox Film Corporation and Twentieth Century Pictures in 1935. In 1985, the studio was acquired by News Corporation, which was succeeded by 21st Century Fox in 2013 following the spin-off of its publishing assets. In 2019, Disney acquired 20th Century Fox through its merger with 21st Century Fox.
1211 Avenue of the Americas (also known as the News Corp. Building) is an International style skyscraper in Midtown Manhattan, New York City. Formerly called the Celanese Building, it was completed in 1973 as part of the Rockefeller Center extension, that started in the late 1950s with the Time-Life Building. The Celanese Corporation would later move to Dallas, Texas. Currently, 1211 is owned by an affiliate of Beacon Capital Partners, and leasing is managed by Cushman & Wakefield, Inc., of which the Rockefeller Group was once a major shareholder. The structure has a simple slab-like shape devoid of any decoration; its prosaic façade consisting of vertical alternating limestone and glass stripes. The façade stone piers are supernumerary; there are twice as many of them as structurally necessary. The glass bands are continuous and offer no indication of floor levels. These features ably create the visual lack of scale, so the tower does not look overly bulky.
, answer the query:What is the headquarters location of the sequel of of the owner of the distributed by of the day the earth stood still?. Answer 2-5 words.</s>"""

In [157]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# input_ids = tokenizer1.apply_chat_template(messages1, return_tensors="pt")
input_ids.shape
torch.cuda.empty_cache()
gc.collect()
model_inputs = input_ids.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens=20, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids.detach().cpu())

In [160]:
tokenizer.batch_decode(generated_ids[:, model_inputs.shape[-1]:])

['1.\n\nAnswer 1: Joseph Paterno died in State College, Pennsylvania, United States']

In [151]:
prompt

'Given context: ICDC College was a private for-profit, post-secondary education career college. ICDC College had Four Southern California campuses (Van Nuys, South Bay, Hollywood, and Huntington Park) and an online campus.  The college closed in 2016 and Trident University International has agreed to continue instruction of ICDC\'s current academic programs.icdc college is also known as the day the earth stood still.\nTwenty-First Century Fox, Inc., doing business as 21st Century Fox (21CF), was an American multinational mass media corporation that was based in Midtown Manhattan, New York City. It was one of the two companies formed from the 2013 spin-off of the publishing assets of News Corporation, as founded by Rupert Murdoch in 1980.21st Century Fox was the legal successor to News Corporation dealing primarily in the film and television industries. It was the United States\' fourth-largest media conglomerate until its acquisition by The Walt Disney Company in 2019. The other compan

In [152]:
decoded[0]

'<s> Given context: ICDC College was a private for-profit, post-secondary education career college. ICDC College had Four Southern California campuses (Van Nuys, South Bay, Hollywood, and Huntington Park) and an online campus.  The college closed in 2016 and Trident University International has agreed to continue instruction of ICDC\'s current academic programs.icdc college is also known as the day the earth stood still.\nTwenty-First Century Fox, Inc., doing business as 21st Century Fox (21CF), was an American multinational mass media corporation that was based in Midtown Manhattan, New York City. It was one of the two companies formed from the 2013 spin-off of the publishing assets of News Corporation, as founded by Rupert Murdoch in 1980.21st Century Fox was the legal successor to News Corporation dealing primarily in the film and television industries. It was the United States\' fourth-largest media conglomerate until its acquisition by The Walt Disney Company in 2019. The other co

In [156]:
decoded[0][-150:]

'y: What is the location of death of joseph paterno?. Start with answer 1.\n\nAnswer 1: Joseph Paterno died in State College, Pennsylvania, United States'

In [101]:
dict1['correct_answer']

'pennsylvania'

In [62]:
decoded[0][-100:]

'ation of death of joseph paterno?. Start with the answer in one word or phrase. 1. 2. 3. 4. 5. 6. 7.'

In [44]:
some_dict = """{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% else %}{% set loop_messages = messages %}{% set system_message = 'A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user\\'s questions.' %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if loop.index0 == 0 %}{{ system_message }}{% endif %}{% if message['role'] == 'user' %}{{ ' USER: ' + message['content'].strip() }}{% elif message['role'] == 'assistant' %}{{ ' ASSISTANT: ' + message['content'].strip() + eos_token }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ ' ASSISTANT:' }}{% endif %}"""

In [69]:
del model_inputs, generated_ids, decoded

In [19]:


pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0, max_new_tokens=20, temperature=0)

In [24]:
prompt = """### Human: Given a question: Which of the following best describes the structure that collects urine in the body? and a correct answer: Bladder, is there a way to generate a word same as correct answer but not verbatim?
### Assistant:"""
ans = pipe(prompt)[0]['generated_text']
ans

'### Human: Given a question: What is the Capital of France? and a correct answer: Paris, is there a way to generate a word same as correct answer but not verbatim?\n### Assistant: Yes, there are several ways to generate a word that is similar to the correct answer but not verb'